## 時系列を考慮した適切なモデル評価

データリークを防ぎ、実際の運用を想定した評価を行います。

In [3]:
import sys
sys.path.append('src')

from models.proper_evaluation import ProperModelEvaluator
import warnings
warnings.filterwarnings('ignore')

ModuleNotFoundError: No module named 'lightgbm'

### 1. 単一期間での評価

2020-2021年のデータで学習し、2022年で検証、2023年でテストします。

In [ ]:
evaluator = ProperModelEvaluator()

# 時系列検証の実行
result = evaluator.run_time_series_validation(
    train_years=[2020, 2021],  # 学習データ
    valid_year=2022,           # 検証データ（閾値決定用）
    test_year=2023             # テストデータ（最終評価用）
)

### 2. ウォークフォワード分析

実際の運用を想定し、過去データのみを使って順次予測を行います。

In [ ]:
# 2019年から2023年まで、2年間の訓練データで順次予測
results = evaluator.run_walk_forward_analysis(
    start_year=2019,
    end_year=2023,
    train_window=2  # 過去2年分のデータで学習
)

### 3. 特徴量重要度の確認

最新のモデルで、どの特徴量が重要かを確認します。

In [ ]:
# 最新モデルの特徴量重要度
if results:
    latest_model = results[-1]['model']
    feature_importance = latest_model.feature_importances_
    feature_names = latest_model.feature_name_
    
    # 重要度でソート
    import numpy as np
    indices = np.argsort(feature_importance)[::-1]
    
    print("Top 20 Important Features:")
    for i in range(min(20, len(indices))):
        idx = indices[i]
        print(f"{i+1:2d}) {feature_names[idx]:<30} {feature_importance[idx]:>6}")

### 4. 結果の可視化

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

# 年ごとの回収率をプロット
if results:
    years = [r['test_year'] for r in results]
    win_rates = [r['return_results']['win_return_rate'] for r in results]
    place_rates = [r['return_results']['place_return_rate'] for r in results]
    aucs = [r['test_auc'] for r in results]
    
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(10, 8))
    
    # 回収率のプロット
    ax1.plot(years, win_rates, 'o-', label='単勝回収率', linewidth=2)
    ax1.plot(years, place_rates, 's-', label='複勝回収率', linewidth=2)
    ax1.axhline(y=100, color='r', linestyle='--', label='損益分岐点')
    ax1.set_xlabel('年')
    ax1.set_ylabel('回収率 (%)')
    ax1.set_title('年ごとの回収率推移')
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    
    # AUCのプロット
    ax2.plot(years, aucs, 'o-', color='green', linewidth=2)
    ax2.set_xlabel('年')
    ax2.set_ylabel('AUC')
    ax2.set_title('モデル性能（AUC）の推移')
    ax2.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # 結果のサマリーテーブル
    summary_df = pd.DataFrame({
        'Year': years,
        'AUC': aucs,
        'Bets': [r['return_results']['bet_count'] for r in results],
        'Win Return (%)': win_rates,
        'Place Return (%)': place_rates
    })
    
    print("\nSummary Table:")
    print(summary_df.to_string(index=False))

### 5. 現実的な期待値

データリークを排除した場合の現実的な回収率は：
- **単勝**: 70-90%程度
- **複勝**: 80-95%程度

100%を超えることは稀で、長期的に利益を出すのは非常に困難です。